In [212]:
import pandas as pd
import numpy as np

from functools import reduce

This problem in math is defined as finding [partitions][1] of 100. 

A simple start is to generate all lists for each sum, and use recursion and a cache to optimize slightly. However, even for $n = 100$, this is far too slow. So then we use a dynamic programming approach.

The idea of the approach is that for each integer $i$, if it has more partitions, it contributes to all future integers $j$. This is reflected in the double for loop, where we are considering the partitions of $i$ in the outer loop, and in the inner loop we update each future partition based on all previously completed partitions.

So in iteration $i = 1$, $j - i$ lags behind $j$ by $1$. So this is showing the effect of each $j-1$ partitions on the $j$ partitions. In iteration $i = k$, $j - i$ lags behind $j$ by $k$. So this is showing the effect of the $j-k$ partitions on the $j$ partitions.

  [1]: https://en.wikipedia.org/wiki/Partition_(number_theory)

In [236]:
def partition(n):
    # initialize partitions
    partitions = [1] + [0]*n

    # for each iteration
    for i in range(1, n+1):
        # for every later partition
        for j in range(i, n+1):
            # how do previous partitions affect future partitions
            partitions[j] += partitions[j - i]

    # adjustment since we are only looking for solutions with >= 2 numbers
    return np.array(partitions) - np.ones(n+1, int)

n = 10**4
print(partition(n)[n])

36167251325636293988820471890953695495016030339315650422081868605887952568754066420592310556052906916435143


If we want to improve further, it would be better, for large $n$ (for example $ n = 10,\!000$), to use a formulation based on [Euler's pentagonal theorem][1].

  [1]: https://en.wikipedia.org/wiki/Pentagonal_number_theorem#Relation_with_partitions

In [235]:
def partition_pentagonal(n):
    # intialize partitions
    partitions = [1] + [0]*n

    # initialzie pentagonal numbers and signs for the summation
    pentagonals = [0]
    signs = [0]
    k, sign = 1, 1
    while pentagonals[-1] < n:
        pentagonals.append(k*(3*k - 1) // 2)
        pentagonals.append(k*(3*k + 1) // 2)
        signs += [sign, sign]
        k += 1
        sign *= -1
    
    for x in range(1, n+1):
        # implement Euler's pentagonal number theorem
        # p(n) = sum(k) signs(k) * partition(x - pentagonals(k))
        # we can stop when x - pentagonals(k) < 0, since partition(n) = 0 when n < 0
        k = 1
        while pentagonals[k] <= x:
            partitions[x] += partitions[x - pentagonals[k]] * signs[k]
            k += 1
            if k >= len(pentagonals):
                break

    # adjustment since we are only looking for solutions with >= 2 numbers
    return np.array(partitions) - np.ones(n+1, int)

n = 10**4
print(partition_pentagonal(n)[n])

36167251325636293988820471890953695495016030339315650422081868605887952568754066420592310556052906916435143
